In [1]:
# dependencies
import pymongo
import pandas as pd
import json

In [2]:
# setup
client = pymongo.MongoClient()
database = client.fds_twitter
collection = database.messages
collection.create_index([('user.id', 1)])

'user.id_1'

In [32]:
users = collection.find(projection=["user.id_str", "in_reply_to_status_id_str", "in_reply_to_user_id_str"])
df = pd.DataFrame(users)
df

,_id,in_reply_to_status_id_str,in_reply_to_user_id_str,user
0,5d7e1415af475bc1f98581ad,764038820476051456,10774652,{'id_str': '366636488'}
1,5d7e1415af475bc1f98581ae,None,813286,{'id_str': '82496193'}
2,5d7e1415af475bc1f98581af,764038820476051456,10774652,{'id_str': '366636488'}
3,5d7e1415af475bc1f98581b0,762623255005966336,1339835893,{'id_str': '44032624'}
4,5d7e1415af475bc1f98581b1,None,None,{'id_str': '769208504'}
...,...,...,...,...
657302,5d7e212caf475bc1f98f8943,None,None,{'id_str': '569487350'}
657303,5d7e212caf475bc1f98f8944,None,126629610,{'id_str': '716017946166857728'}
657304,5d7e212caf475bc1f98f8945,None,None,{'id_str': '44489439'}
657305,5d7e212caf475bc1f98f8946,None,None,{'id_str': '3241116564'}


In [33]:
df['user'] = df['user'].apply(lambda x: str(x['id_str']))

In [34]:
# remove all rows that are not a reply
df = df.dropna()
df

,_id,in_reply_to_status_id_str,in_reply_to_user_id_str,user
0,5d7e1415af475bc1f98581ad,764038820476051456,10774652,366636488
2,5d7e1415af475bc1f98581af,764038820476051456,10774652,366636488
3,5d7e1415af475bc1f98581b0,762623255005966336,1339835893,44032624
9,5d7e1415af475bc1f98581b6,764037223503331328,11966392,740623508
10,5d7e1415af475bc1f98581b7,764001459671150595,4852163069,1507953240
...,...,...,...,...
657292,5d7e212caf475bc1f98f8939,775322944209883136,3001269268,3001269268
657293,5d7e212caf475bc1f98f893a,775320576911024128,3293178390,3293178390
657299,5d7e212caf475bc1f98f8940,775008958364852224,377751695,242992653
657300,5d7e212caf475bc1f98f8941,775322485105500160,332888709,332888709


In [38]:
grouped = df.groupby('user')['in_reply_to_user_id_str'].agg(list)

In [49]:
grouped

user
100007790                                           [274218280]
1000153826                                           [25073877]
100025085                           [1367531, 9300262, 1367531]
1000340761                              [127375220, 1339835893]
100041840     [25073877, 25073877, 25073877, 25073877, 56205...
                                    ...                        
999576338                                            [39689218]
99959281      [116245566, 2468758092, 705555762538487808, 99...
99960562      [25073877, 809887021, 24578794, 25073877, 2542...
999642914                                              [813286]
99981345                                            [489596897]
Name: in_reply_to_user_id_str, Length: 39604, dtype: object

In [50]:
grouped.to_csv('../datasets/user_replies.csv')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
